In [2]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm

print(f"Running on PyMC3 v{pm.__version__}")

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Running on PyMC3 v3.11.2


In [33]:
#df = pd.read_csv('/Users/sletkeman/Downloads/HW5/hadlock.csv')
df = pd.read_csv("./train.csv")
df['ones'] = 1
matrix = df[['ones', 'GDP', 'Adult.Mortality', 'BMI', 'Diphtheria' ,'HIV.AIDS', 'Income.composition.of.resources', 'Schooling']].to_numpy()

In [38]:
n = len(df.index)

model = pm.Model()

with model:
    tau_likelihood = pm.Gamma("tau", alpha=0.001, beta=0.001)

    #tau_beta = n * np.linalg.inv(matrix.T.dot(matrix)) / tau_likelihood

    # Priors for unknown model parameters
    beta = pm.Normal("beta", mu=0, tau=0.00001, shape=8)


    # Expected value of outcome
    mu = beta[0] \
      + beta[1] * df['GDP'].values \
      + beta[2] * df['Adult.Mortality'].values \
      + beta[3] * df['BMI'].values \
      + beta[4] * df['Diphtheria'].values \
      + beta[5] * df['HIV.AIDS'].values \
      + beta[6] * df['Income.composition.of.resources'].values \
      + beta[7] * df['Schooling'].values



    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal("Y_obs", mu=mu, tau=tau_likelihood, observed=df['Life.expectancy'].values)

In [39]:
SAMPLES = 1000
BURN = 100
with model:
    step = pm.Metropolis()
    trace = pm.sample(SAMPLES, step, chains=2, tune=BURN)
    display(az.summary(trace, round_to=4))

<ipython-input-39-0f387c8fbfb0>:5: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(SAMPLES, step, chains=2, tune=BURN)
Multiprocess sampling (2 chains in 4 jobs)
CompoundStep
>Metropolis: [beta]
>Metropolis: [tau]


Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 234 seconds.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],-0.3970,0.7713,-1.8949,0.0000,0.4966,0.4054,2.4123,2.4123,2.4820
beta[1],-0.0003,0.0005,-0.0012,0.0000,0.0003,0.0003,2.4123,2.4123,2.4820
beta[2],-0.0491,0.0954,-0.2343,0.0000,0.0614,0.0501,2.4123,2.4123,2.4820
beta[3],0.2232,0.4336,0.0000,1.0652,0.2792,0.2279,2.4123,2.4123,2.4820
beta[4],0.1407,0.2734,0.0000,0.6716,0.1760,0.1437,2.4123,2.4123,2.4820
beta[5],0.1463,0.2843,0.0000,0.6983,0.1830,0.1494,2.4123,2.4123,2.4820
beta[6],-0.3762,0.7310,-1.7959,0.0000,0.4707,0.3842,2.4123,2.4123,2.4820
beta[7],-0.0225,0.0436,-0.1072,0.0000,0.0281,0.0229,2.4123,2.4123,2.4820
tau,0.0003,0.0001,0.0002,0.0005,0.0001,0.0000,4.9703,106.4701,1.4609


In [ ]:
    mu = beta[0] \
      + beta[1] * df['GDP'].values \
      + beta[2] * df['Adult.Mortality'].values \
      + beta[3] * df['BMI'].values \
      + beta[4] * df['Diphtheria'].values \
      + beta[5] * df['HIV.AIDS'].values \
      + beta[6] * df['Income.composition.of.resources'].values \
      + beta[7] * df['Schooling'].values